# 设置 Cookie

本笔记讲解如何在 Python 中使用 requests 管理 HTTP Cookies：如何设置、读取、持久化和常见注意点，包含可运行示例（基于 httpbin.org 回显服务）。

## 目录
1. Cookie 基础概念（简要）
2. requests 中设置和读取 Cookies（简单示例）
3. 使用 Session 管理持久连接和 cookies
4. cookiejar（持久化 cookies 到文件）
5. 手动在 Header 中设置 Cookie
6. Domain / Path / Secure / HttpOnly / SameSite 等注意点
7. 常见坑与调试技巧


## 1. Cookie 基础概念（简要）
- Cookie 是服务器通过 Set-Cookie 响应头下发给客户端的数据，用于维持会话等状态。
- Cookie 带有属性：domain、path、expires/max-age、secure、HttpOnly、SameSite 等。
- 浏览器会根据这些属性决定是否在后续请求中携带 Cookie；requests 模拟客户端行为，但在某些细节（例如 SameSite）上需要手动管理。

## 2. requests 中设置和读取 Cookies（简单示例）
- 你可以通过 `cookies` 参数在单次请求中发送 cookie（传入 dict）。
- 响应对象 `Response` 有 `.cookies` 属性，可读取服务器设置的 cookies（一个 RequestsCookieJar）。

In [1]:
import requests

# 直接在单个请求中发送 cookies
cookies = {'sessionid': '12345', 'pref': 'zh'}
r = requests.get('https://httpbin.org/cookies', cookies=cookies, timeout=10)
print('请求 URL:', r.url)
print('响应 cookies (服务器视角):', r.json())

# 读取服务器返回的 Set-Cookie（转换为字典）
resp = requests.get('https://httpbin.org/cookies/set?token=abc', timeout=10)
print('响应 Set-Cookie 后的 cookies:', resp.cookies.items())


请求 URL: https://httpbin.org/cookies
响应 cookies (服务器视角): {'cookies': {'pref': 'zh', 'sessionid': '12345'}}
响应 Set-Cookie 后的 cookies: []


## 3. 使用 Session 管理持久连接和 cookies
- requests.Session 会在同一个 session 对象中自动保存 cookies，适合需要在多个请求间保持登录态的场景。
- Session 还可以复用 TCP 连接，提高性能。

In [2]:
# 使用 Session 示例：先设置 cookie，再在后续请求中自动带上
from requests import Session

s = Session()
# 访问 /cookies/set 会让 server 在响应中设置 cookie，Session 会保存
r1 = s.get('https://httpbin.org/cookies/set?name=value', timeout=10)
print('设置后，session cookies:', s.cookies.get_dict())
# 再次访问 /cookies 将会自动携带已保存的 cookie
r2 = s.get('https://httpbin.org/cookies', timeout=10)
print('回显 cookies:', r2.json())


设置后，session cookies: {'name': 'value'}
回显 cookies: {'cookies': {'name': 'value'}}


## 4. cookiejar：持久化 cookies 到文件
- Python 标准库提供 `http.cookiejar` 的实现（如 `MozillaCookieJar`, `LWPCookieJar`），可以把 cookies 保存到文件并在之后加载。
- requests 支持与 cookiejar 互转（requests.utils.cookiejar_from_dict / cookiejar_to_dict），并且 session.cookies 本身是兼容 cookiejar 的对象。

In [3]:
import os
from http.cookiejar import MozillaCookieJar
from requests import Session

cookie_file = 'cookies.txt'
s = Session()
# 让 session 保存到一个 Mozilla 格式的文件（如果文件不存在会创建）
jar = MozillaCookieJar(cookie_file)
# 把 jar 绑定到 session（注意：Session.cookies 是 RequestsCookieJar，下面步骤为示例）
s.cookies = jar
# 通过 httpbin 设置 cookie 并保存
try:
    s.get('https://httpbin.org/cookies/set?persist=1', timeout=10)
    # 保存到文件
    jar.save(ignore_discard=True, ignore_expires=True)
    print('已将 cookies 保存到', os.path.abspath(cookie_file))
except Exception as e:
    print('保存 cookies 时出现问题（可能是网络）：', e)

# 以后可以用 jar.load() 恢复（需要在 session.cookies 前绑定 jar）


已将 cookies 保存到 C:\Users\ocean\Desktop\fly-spiders\notes\cookies.txt


## 5. 手动在 Header 中设置 Cookie
- 有时你需要手工构造 Cookie 请求头（例如从浏览器复制过来的 cookie 字符串）。可以直接在 `headers['Cookie']` 中放置。
- **注意：直接设置 headers 的 Cookie 会覆盖 session 自动处理的行为（手动优先）。**

In [4]:
# 手动在 headers 中设置 Cookie 示例
headers = {'Cookie': 'a=1; b=2'}
r = requests.get('https://httpbin.org/headers', headers=headers, timeout=10)
print('请求头回显包含 Cookie:', 'Cookie' in r.json().get('headers', {}))
print('回显的 Cookie 字符串:', r.json().get('headers', {}).get('Cookie'))


请求头回显包含 Cookie: True
回显的 Cookie 字符串: a=1; b=2


## 6. Domain / Path / Secure / HttpOnly / SameSite 等注意点
- domain：Cookie 只能被指定域名及其子域访问（例如 domain=example.com 可被 a.example.com 访问）。
- path：控制哪些路径可以发送该 cookie。
- secure：只有在 HTTPS 请求中才会发送该 cookie。
- HttpOnly：浏览器 JS 无法读取，但 HTTP 客户端（如 requests）依然可以在请求中发送，requests 无法直接检测 HttpOnly 与否（cookie 信息来自服务器响应头）
- SameSite：限制跨站请求时是否发送 cookie（Lax/Strict/None）。requests 作为脚本客户端不会自动遵守浏览器的 SameSite 策略，如果你需要模拟浏览器行为需手动过滤。

## 7. 常见坑与调试技巧
- Cookie 没被发送：检查域名/路径是否匹配、是否为 HTTPS（secure）。
- 二次编码或错误的 header：优先使用 session 或 cookies 参数，而不是手工拼接不合规的字符串。
- HttpOnly 与 JavaScript：浏览器 JS 不能读取 HttpOnly，但 requests 可在请求中发送来自服务器的 HttpOnly cookie（因为它从响应头中读到并保存）。
- 日志与安全：不要将敏感 cookie（如 session token）写到不可控的日志或公开 repo。
- 调试工具：使用抓包（Fiddler、Charles），或在 httpbin.org 的回显端点来观察 server 视角下的 cookie。